In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
from sklearn.metrics import mean_absolute_error

In [3]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
test_data.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [7]:
train_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [8]:
train_data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [9]:
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [10]:
train_data[train_data.Age.isna()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,NaN,Q
...,...,...,...,...,...,...,...,...,...,...,...,...
859,860,0,3,"Razi, Mr. Raihed",male,NaN,0,0,2629,7.2292,NaN,C
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,CA. 2343,69.5500,NaN,S
868,869,0,3,"van Melkebeke, Mr. Philemon",male,NaN,0,0,345777,9.5000,NaN,S
878,879,0,3,"Laleff, Mr. Kristo",male,NaN,0,0,349217,7.8958,NaN,S


In [11]:
train_data.groupby('Pclass').mean()['Age']

Pclass
1    38.233441
2    29.877630
3    25.140620
Name: Age, dtype: float64

In [12]:
train_data.groupby('Pclass').mean()['Age'][3]

25.14061971830986

In [13]:
def age_approx(cols):
    Age = cols[0]
    Pclass = cols[1]
    if pd.isnull(Age):
        return train_data.groupby('Pclass').mean()['Age'][Pclass]
    else: 
        return Age

In [14]:
train_data['Age'] = train_data[['Age','Pclass']].apply(age_approx, axis=1)
test_data['Age'] = test_data[['Age','Pclass']].apply(age_approx, axis=1)

In [15]:
train_data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [16]:
test_data.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [17]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

# print(women)
print("% of women who survived:", rate_women)

% of women who survived: 0.7420382165605095


In [18]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

# print(men)
print("% of men who survived:", rate_men)

% of men who survived: 0.18890814558058924


In [19]:
# Don't run this. Else the competition will not accept lesser predicted values.
# test_data.dropna(subset=['Age'], axis=0, inplace=True)

In [20]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
mm = MinMaxScaler()

train_data['Age'] = mm.fit_transform(train_data[['Age']])
test_data['Age'] = mm.fit_transform(test_data[['Age']])

In [21]:
y = train_data["Survived"]
features = ["Pclass", "Sex", "SibSp", "Parch", "Age"]

X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

# model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
# model.fit(X, y)
# predictions = model.predict(X_test)

# output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
# output.to_csv('submission.csv', index=False)
# print("Your submission was successfully saved!")

In [22]:
X.describe()

,Pclass,SibSp,Parch,Age,Sex_female,Sex_male
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,2.308642,0.523008,0.381594,0.362816,0.352413,0.647587
std,0.836071,1.102743,0.806057,0.166003,0.477990,0.477990
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,0.271174,0.000000,0.000000
50%,3.000000,0.000000,0.000000,0.321438,0.000000,1.000000
75%,3.000000,1.000000,0.000000,0.459663,1.000000,1.000000
max,3.000000,8.000000,6.000000,1.000000,1.000000,1.000000


In [23]:
X.head()

,Pclass,SibSp,Parch,Age,Sex_female,Sex_male
0,3,1,0,0.271174,0,1
1,1,1,0,0.472229,1,0
2,3,0,0,0.321438,1,0
3,1,1,0,0.434531,1,0
4,3,0,0,0.434531,0,1


In [24]:
X_test.isna().sum()

Pclass        0
SibSp         0
Parch         0
Age           0
Sex_female    0
Sex_male      0
dtype: int64

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

# Decision Tree

In [26]:
from sklearn.tree import DecisionTreeRegressor
model_d = DecisionTreeRegressor(random_state=1)
model_d.fit(X_train, y_train)

pred_d_train = model_d.predict(X_train)
pred_d_test = model_d.predict(X_valid)

print('TRAIN DATA')
print("MAE in Decision Tree: %.4f" % mean_absolute_error(pred_d_train, y_train))
print('R-Squared in Decision Tree: %.2f' % model_d.score(X_train, y_train))

print('\nTEST DATA')
print('MAE in Decision Tree: %.4f' % mean_absolute_error(pred_d_test, y_valid))
print('R-Squared in Decision Tree: %.2f' % model_d.score(X_valid, y_valid))


TRAIN DATA
MAE in Decision Tree: 0.0961
R-Squared in Decision Tree: 0.79

TEST DATA
MAE in Decision Tree: 0.2325
R-Squared in Decision Tree: 0.22


# ****Random Forest****

In [27]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model_rf.fit(X_train, y_train)

pred_rf_train = model_rf.predict(X_train)
pred_rf_test = model_rf.predict(X_valid)

print('TRAIN DATA')
print("MAE in Random Forest Model: %.4f" % mean_absolute_error(pred_rf_train, y_train))
print('R-Squared in Random Forest Model: %.2f' % model_rf.score(X_train, y_train))

print('\nTEST DATA')
print('MAE in Random Forest Model: %.4f' % mean_absolute_error(pred_rf_test, y_valid))
print('R-Squared in Random Forest Model: %.2f' % model_rf.score(X_valid, y_valid))

TRAIN DATA
MAE in Random Forest Model: 0.1587
R-Squared in Random Forest Model: 0.84

TEST DATA
MAE in Random Forest Model: 0.1749
R-Squared in Random Forest Model: 0.83


# XGB

In [28]:
from xgboost import XGBRegressor
model_xgb = XGBRegressor(n_estimators=1000, n_iter_no_change=5, learning_rate=0.5)

model_xgb.fit(X_train, y_train, verbose=False)

pred_xgb_train = model_xgb.predict(X_train)
pred_xgb_test = model_xgb.predict(X_valid)

print('TRAIN DATA')
print("MAE in XGB Model: %.4f" % mean_absolute_error(pred_xgb_train, y_train))
print('R-Squared in XGB Model: %.2f' % model_xgb.score(X_train, y_train))

print('\nTEST DATA')
print('MAE in XGB Model: %.4f' % mean_absolute_error(pred_xgb_test, y_valid))
print('R-Squared in XGB Model: %.2f' % model_xgb.score(X_valid, y_valid))

[10:13:06] WARNING: ../src/learner.cc:627: 
Parameters: { "n_iter_no_change" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


TRAIN DATA
MAE in XGB Model: 0.0993
R-Squared in XGB Model: 0.79

TEST DATA
MAE in XGB Model: 0.2736
R-Squared in XGB Model: 0.24


# Logistic Regression

In [29]:
from sklearn.linear_model import LogisticRegression
model_lg = LogisticRegression(random_state=1)

model_lg.fit(X_train, y_train)

pred_lg_train = model_lg.predict(X_train)
pred_lg_test = model_lg.predict(X_valid)

print('TRAIN DATA')
print("MAE in Logistic Regression Model: %.4f" % mean_absolute_error(pred_lg_train, y_train))
print('R-Squared in Logistic Regression Model: %.2f' % model_lg.score(X_train, y_train))

print('\nTEST DATA')
print('MAE in Logistic Regression Model: %.4f' % mean_absolute_error(pred_lg_test, y_valid))
print('R-Squared in Logistic Regression Model: %.2f' % model_lg.score(X_valid, y_valid))

TRAIN DATA
MAE in Logistic Regression Model: 0.1871
R-Squared in Logistic Regression Model: 0.81

TEST DATA
MAE in Logistic Regression Model: 0.2063
R-Squared in Logistic Regression Model: 0.79


# Submission

In [30]:
model_lg.fit(X, y)
predictions = model_lg.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
